In [179]:
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Input
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn import metrics
import time

In [180]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [181]:
train_data_dir = "D:\chest_xray\\test"
test_data_dir =  "D:\chest_xray\\train"
    
validation_data_dir = "D:\chest_xray\\val"


In [182]:
img_width, img_height = 150, 150
nb_train_samples = 2000
nb_validation_samples = 800
batch_size = 16 
epochs = 5
test_generator_samples = 300
test_batch_size = 24
input_shape = (img_width, img_height, 3)


In [242]:
model = Sequential()

In [243]:
model.add( Input(shape = (img_width,img_height, 3) ))

In [244]:
model.add(Conv2D(filters=64, kernel_size=3, strides= 2, padding='same',activation= 'relu' ))

In [245]:
model.add(MaxPool2D())

In [246]:
model.add(Conv2D(filters=32,kernel_size=3,activation= 'relu' ))

In [247]:
model.add(MaxPool2D())

In [248]:
model.add(Conv2D(filters=32,kernel_size=3,activation= 'relu' ))

In [249]:
model.add(MaxPool2D())

In [250]:
model.add(Dense(64, activation='relu'))

In [251]:
model.add(Dropout(rate=0.5))

In [252]:
model.add(Flatten(name = "FlattenedLayer"))

In [253]:
model.add(Dense(1, activation='sigmoid'))

In [254]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 75, 75, 64)        1792      
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 37, 37, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_25 (Conv2D)          (None, 35, 35, 32)        18464     
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 17, 17, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 15, 15, 32)        9248      
                                                                 
 max_pooling2d_26 (MaxPoolin  (None, 7, 7, 32)       

In [255]:
model.compile(loss='binary_crossentropy', optimizer= 'adam', metrics=['accuracy'])

In [256]:
tr_dtgen = ImageDataGenerator(
                              rescale=1. / 255,     
                              shear_range=0.2,      
                              zoom_range=0.2,
                              horizontal_flip=True,
                              )

In [257]:
train_generator = tr_dtgen.flow_from_directory(
                                               train_data_dir,       
                                               target_size=(img_width, img_height),
                                               batch_size=batch_size,
                                               class_mode='binary'   
                                                )


Found 624 images belonging to 2 classes.


In [258]:
val_dtgen = ImageDataGenerator(rescale=1. / 255)

In [259]:
validation_generator = val_dtgen.flow_from_directory(
                                                     validation_data_dir,
                                                     target_size=(img_width, img_height),   
                                                     batch_size=batch_size,    
                                                     class_mode='binary'  
                                                     )

Found 16 images belonging to 2 classes.


In [266]:
start = time.time()
history = model.fit_generator(
                              generator = train_generator,
                              steps_per_epoch=nb_train_samples // batch_size,
                              epochs=2,
                              validation_data=validation_generator,
                              verbose = 1,
                              validation_steps=nb_validation_samples // batch_size
                              )

end = time.time()
(end - start)/60

Epoch 1/2


C:\Users\ubuntu\AppData\Local\Temp\ipykernel_83276\665502559.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


125/125 [==============================] - 8s 60ms/step - loss: 0.4533 - accuracy: 0.8029 - val_loss: 0.7125 - val_accuracy: 0.6250


0.13008796374003093

In [261]:
result = model.evaluate(
                        validation_generator,
                        verbose = 1,
                        steps = 4        
                        )
result

4/4 [==============================] - 0s 7ms/step - loss: 0.6981 - accuracy: 0.5000


[0.6980636119842529, 0.5]

In [262]:
pred = model.predict(validation_generator, steps = 2)
pred[:5] 

2/2 [==============================] - 0s 11ms/step


array([[0.54292583],
       [0.5595155 ],
       [0.56494415],
       [0.5606516 ],
       [0.56146365]], dtype=float32)